# NFL Data Bowl -- Consolidating and Merging Data

This notebook will go through the methodology of data collection, cleaning, and return a finished dataset of a narrowed-down analysis to passing/running plays in the 2018 NFL season. The data is obtained from the NFL's 2021 Big Data Bowl: https://www.kaggle.com/c/nfl-big-data-bowl-2021/

*There is potential to combine the 2020 data bowl data, which contains similar info to 2021 data bowl data except about rushing plays 2017-2019. Combining these sources to produce a similar notebook to the original tendency analysis. Less data, but more information in our columns.*

The main focus of this analysis is to see how offensive / defensive personnel and the formation and defensive players on the field affect the decision to run or pass the ball. This can be a unique opportunity to utilize tracking / location data of players as well (which may be explored in a separate notebook).

Other data bowls for reference:
- https://www.kaggle.com/c/nfl-big-data-bowl-2020: Forecast yardage gained on the run plays
- https://www.kaggle.com/c/nfl-big-data-bowl-2022: Analyze special teams data
- https://github.com/nfl-football-ops/Big-Data-Bowl: Inaugural data bowl from 2019, useful R code on animation of tracking

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
bdb_pass = pd.read_csv('nfl-big-data-bowl/pass-2018.csv')
bdb_rush = pd.read_csv('nfl-big-data-bowl/rush.csv')

C:\Users\tshah\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (47) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# Useful Features Present in Both Datasets

Before cleaning the data, from a cursory examination of our data, the main features I think will be useful in this analysis are the following. We can only use pre-snap features in this analysis, or else it defeats the purpose.

- Type of play (run/pass): TARGET (need to create)
- Drive number (need to create / merge with other dataset)
- Offensive / defensive team (`HomeTeamAbbr`, `VisitorTeamAbbr`, `PossessionTeam`, need to create `DefTeam`)
- Quarter of the game (`Quarter`)
- Down number (`Down`)
- Time left in a quarter (`GameClock`, need to format this)
- Yards to gain for a first down (`Distance`)
- Yards to gain for a touchdown (100 minus current yardline position, `YardLine`)
- Current score in the game (model as difference, `HomeScoreBeforePlay - VisitorScoreBeforePlay`)
- Offensive formation (`OffenseFormation`)
- Offensive personnel (`OffensePersonnel`)
- Defenders in the box (`DefendersInTheBox`)
- Defensive personnel (`DefensePersonnel`)
- Week of season (`Week`)

# Useful Features Present Only in Rushing Data

Some features are only included in the rushing play data. However, for 2018 alone, we could match up the games and include further information in the analysis for pass plays as well.

- Stadium type (`StadiumType`)
- Turf or grass (`Turf`)
- Weather in game (`GameWeather`)
- Temperature on game day (`Temperature`)
- Humidity on game day (`Humidity`)
- Wind speed on game day (`WindSpeed`)

## Next Steps / Preparing Data

- Is it more useful to do an analysis on an imbalanced dataset with rushing 2017-2019 and passing 2018, with only the first subset of features?
- Or would I rather narrow down the rushing plays to only 2018, and use all features (can match up `GameID` and thus find the weather/stadium data by game for the passing plays as well)
- Aside from that, still need to complete the following data cleaning steps regardless of which direction I choose:
    - Merge both datasets into same format
    - Ignore post-snap outcomes (i.e. yardage gained, direction of the run, play results, type of pass dropback)
    - Ignore tracking data (for now) since all of it is post-snap movements
        - Look into if I can see whether a play is in "motion" i.e. the WR shifting from the X spot to the slot
        - I have the "time of snap" data, so there may be some potential?
    - Change the offensive formation to one-hot encoded columns
    - Change the personnel for offense and defense to one-hot encoded columns
        - Or find a unique way to deal with this since there are many combinations, some with few observations
    - Narrow down each play to a single row
    - More TBD..

# Merging 2018 run/pass

First I will try the simpler case, where we have more features but must narrow our scope to only 2018 plays. We can match these games on `GameID` which will remove run plays in 2017 and 2019. The key issue is that the run data contains many rows for the same play, since the tracking data is baked in. The solution is to remove the tracking data for the rush dataset, effectively leaving one row per play, then merging the two datasets on their `GameID`. I will do this below by grouping on `PlayID` and only retaining information on the play as a whole. Maybe I'll go back to individual players and see if there's an avenue to work with tracking data / pre-snap motion later and add it in as my own feature.

In [3]:
print('Unique pass game data provided: {}'.format(bdb_pass['gameId'].nunique()))
print('Unique rush game data provided: {}   (includes 2017, 2019)'.format(bdb_rush['GameId'].nunique()))
print('Unique pass play data provided: {}'.format(bdb_pass['gameId'].count()))
print('Unique rush play data provided: {} (includes 2017, 2019)'.format(bdb_rush['PlayId'].nunique()))

Unique pass game data provided: 253
Unique rush game data provided: 688   (includes 2017, 2019)
Unique pass play data provided: 19239
Unique rush play data provided: 31007 (includes 2017, 2019)


In [4]:
## Group by play ID, take first since the data we want is all the same
## Don't care for any tracking / individual player data (yet)
bdb_rush_play = bdb_rush.groupby('PlayId').first()

In [5]:
## List of things to drop
to_drop = ['X','Y','S','A','Dis','Dir','Orientation','NflId','DisplayName',
           'JerseyNumber','Season','NflIdRusher','PlayerHeight','PlayerWeight',
           'PlayerBirthDate','PlayerCollegeName','Position', 'PlayDirection',
           'TimeHandoff', 'TimeSnap', 'Yards', 'Team', 'Location', 'WindDirection']
bdb_rush = bdb_rush_play.drop(to_drop, axis=1)
## only 2018 rush plays
bdb_rush_2018 = bdb_rush[bdb_rush['GameId'].astype('str').str[:4] == '2018']
# Create outcome variable
bdb_rush_2018['Type'] = 'run'

In [6]:
print('Unique pass game data provided: {}'.format(bdb_pass['gameId'].nunique()))
print('Unique rush game data provided: {}     (2018)'.format(bdb_rush_2018['GameId'].nunique()))
print('Unique pass play data provided: {}'.format(bdb_pass['gameId'].count()))
print('Unique rush play data provided: {}   (2018)'.format(bdb_rush_2018['GameId'].count()))

Unique pass game data provided: 253
Unique rush game data provided: 256     (2018)
Unique pass play data provided: 19239
Unique rush play data provided: 11271   (2018)


In [7]:
bdb_rush_2018 = bdb_rush_2018.reset_index(drop=True)

In [8]:
# Show columns where we have missing data
bdb_rush_2018.isnull().sum()[bdb_rush_2018.isnull().sum() > 0]

FieldPosition         142
OffenseFormation        2
DefendersInTheBox       1
StadiumType           719
GameWeather           767
Temperature          1002
Humidity              280
WindSpeed            1394
dtype: int64

In [9]:
## FieldPosition
# Upon examination, FieldPosition is None when at 50 yardline, replace None with 'MID'
bdb_rush_2018.loc[bdb_rush_2018['FieldPosition'].isnull(), 'FieldPosition'] = 'MID'

## Don't need to fix OffenseFormation or DefendersInTheBox...it's 3 plays...we can forgo that much
bdb_rush_2018.dropna(subset=['OffenseFormation','DefendersInTheBox'],axis=0, inplace=True)

# The three missing stadiums were MetLife, Stubhub, TIAA Bank Field...all Outdoor (from commented out command below)
# print(bdb_rush_2018.loc[bdb_rush_2018['StadiumType'].isnull(), 'Stadium'].unique())
bdb_rush_2018.loc[bdb_rush_2018['StadiumType'].isnull(), 'StadiumType'] = 'Outdoor'

## When StadiumType is indoor, we dont have values for GameWeather, Temperature, Humidity, Windspeed, or Direction
# First clean up StadiumType to be either indoor or outdoor
bdb_rush_2018['StadiumType'] = ['Indoor' if st=='Retr. Roof - Closed' or st=='Indoors' or st=='Dome' or st=='Domed, closed'
                                         or st=='Retr. Roof-Closed' or st=='Domed' else
                                'Outdoor' if st=='Outdoors' or st=='Open' or st=='Retr. Roof - Open' or st=='Domed, Open'
                                          or st=='Domed, open' or st=='Outside' or st=='Cloudy' or st=='Bowl'
                                          or st=='Retractable Roof'
                                else st for st in bdb_rush_2018['StadiumType']]


## For the rest, no way to easily tell what the true values are for this data...
## First, clean up the categorical values
## GameWeather desired categories: Cloudy, Rain, Clear, Snow, Indoors, NA
bdb_rush_2018['GameWeather'] = ['Cloudy' if gw=='Cloudy and Cool' or gw=='Partly Cloudy' or gw=='Mostly Cloudy'
                                         or gw=='cloudy' or gw=='Cloudy, 50% change of rain' or gw=='Partly cloudy'
                                         or gw=='Partly Clouidy' or gw=='Mostly cloudy' or gw=='Overcast'
                                         or gw=='Cloudy, Rain' or gw=='Hazy' or gw=='Fair' else
                                'Rain'   if gw=='Scattered Showers' or gw=='Rain Chance 40%' or gw=='Light Rain'
                                         or gw=='Rain shower' or gw=='Rainy' else
                                'Clear'  if gw=='Clear and sunny' or gw=='Mostly sunny' or gw=='Sunny and warm'
                                         or gw=='Mostly Sunny' or gw=='Partly Sunny' or gw=='Partly clear'
                                         or gw=='Clear and Sunny' or gw=='Sunny, Windy' or gw=='Clear and Cool'
                                         or gw=='Clear skies' or gw=='Sunny and clear' or gw=='Mostly Sunny Skies'
                                         or gw=='Clear Skies' or gw=='Sunny Skies' or gw=='Clear and cold'
                                         or gw=='Sunny and cold' or gw=='Sunny' else
                                'Indoor' if gw=='N/A (Indoors)' or gw=='N/A Indoor' or gw=='Controlled Climate'
                                         or gw=='Indoors' else
                                None     if gw=='T: 51; H: 55; W: NW 10 mph'
                                else gw for gw in bdb_rush_2018['GameWeather']]

## WindSpeed (is currently a string), extract numbers with regex, keep that as WindSpeed
bdb_rush_2018['WindSpeed'] = bdb_rush_2018['WindSpeed'].astype('str').str.extract('(\d+)')[0].astype('float64')

## While I'm at it, fix categories for Turf: 1 if Turf, 0 if grass
bdb_rush_2018['Turf'] = ['Turf' if t=='Artificial' or t=='Field Turf' or t=='UBU Speed Series-S5-M'
                                or t=='SISGrass' or t=='Twenty-Four/Seven Turf' or t=='FieldTurf360'
                                or t=='A-Turf Titan' or t=='FieldTurf' or t=='Artifical' or t=='FieldTurf 360' else
                         'Grass' for t in bdb_rush_2018['Turf']]

In [10]:
# Show columns where we have missing data after cleaning
bdb_rush_2018.isnull().sum()[bdb_rush_2018.isnull().sum() > 0]
# For these, we cannot really impute values or find this elsewhere
# it accounts for about 5% of our dataset, we will go on with it

GameWeather     809
Temperature    1002
Humidity        280
WindSpeed      1547
dtype: int64

In [11]:
## Drop columns that are post-snap or deterministic for passing data
## Once these are aligned, we can create general features for the whole dataset at once
to_drop = ['playId', 'playDescription', 'typeDropback', 'penaltyCodes', 'penaltyJerseyNumbers',
           'epa', 'isDefensivePI', 'passResult','playType', 'numberOfPassRushers',
           'offensePlayResult', 'playResult', 'absoluteYardlineNumber']
bdb_pass_2018 = bdb_pass.drop(to_drop, axis=1)
# Create outcome variable
bdb_pass_2018['Type'] = 'pass'

In [12]:
## Check missing values for pass data
bdb_pass_2018.isnull().sum()[bdb_pass_2018.isnull().sum() > 0]

yardlineSide           254
offenseFormation       141
personnelO              29
defendersInTheBox       62
personnelD              29
preSnapVisitorScore    639
preSnapHomeScore       639
gameClock              639
dtype: int64

In [13]:
## yardlineSide
# Upon examination, yardlineSide is None when at 50 yardline, replace None with 'MID'
bdb_pass_2018.loc[bdb_pass_2018['yardlineSide'].isnull(), 'yardlineSide'] = 'MID'

## remove rows without Score, Clock, YardlineNumber..those are going to be important, can't find that elsewhere
bdb_pass_2018.dropna(subset=['preSnapVisitorScore'],axis=0, inplace=True)

In [14]:
## Check missing values for pass data after removal
print(bdb_pass_2018.isnull().sum()[bdb_pass_2018.isnull().sum() > 0]) ## still missing OffenseFormation, DefinBox
# After examining some of the results and the respective personnel, these are from fake punts / special teams plays, drop them
# bdb_pass_2018.loc[bdb_pass_2018['offenseFormation'].isnull()]
bdb_pass_2018.dropna(subset=['offenseFormation', 'defendersInTheBox'], axis=0, inplace=True)

offenseFormation     94
defendersInTheBox    16
dtype: int64


In [15]:
print(bdb_pass_2018.isnull().sum()[bdb_pass_2018.isnull().sum() > 0]) ## no more missing

Series([], dtype: int64)


## Now merge the datasets and align columns.

In [16]:
## Match columns between rush and pass
print(bdb_rush_2018.columns)
print(bdb_pass_2018.columns)

Index(['GameId', 'YardLine', 'Quarter', 'GameClock', 'PossessionTeam', 'Down',
       'Distance', 'FieldPosition', 'HomeScoreBeforePlay',
       'VisitorScoreBeforePlay', 'OffenseFormation', 'OffensePersonnel',
       'DefendersInTheBox', 'DefensePersonnel', 'HomeTeamAbbr',
       'VisitorTeamAbbr', 'Week', 'Stadium', 'StadiumType', 'Turf',
       'GameWeather', 'Temperature', 'Humidity', 'WindSpeed', 'Type'],
      dtype='object')
Index(['gameId', 'quarter', 'down', 'yardsToGo', 'possessionTeam',
       'yardlineSide', 'yardlineNumber', 'offenseFormation', 'personnelO',
       'defendersInTheBox', 'personnelD', 'preSnapVisitorScore',
       'preSnapHomeScore', 'gameClock', 'Type'],
      dtype='object')


In [17]:
pass_cols = ['GameId', 'Quarter', 'Down', 'Distance', 'PossessionTeam',
             'FieldPosition', 'YardLine', 'OffenseFormation', 'OffensePersonnel',
             'DefendersInTheBox', 'DefensePersonnel', 'VisitorScoreBeforePlay',
             'HomeScoreBeforePlay', 'GameClock', 'Type']
# assign column names
bdb_pass_2018.columns = pass_cols

## need to merge:
# HomeTeamAbbr, VisitorTeamAbbr, Week, Stadium, StadiumType, Turf,
# GameWeather, Temperature, Humidity, WindSpeed, WindDirection

## need to make this myself, after merging
# Distance From TD, PosTeamScore, DefTeamScore, ScoreDifference, Redzone, Under2Min

In [18]:
## Merge both dataframes, then determine best way to fill in missing data
bdb_2018 = pd.concat([bdb_rush_2018, bdb_pass_2018]).reset_index(drop=True)

In [19]:
## check missing values in merged data
## those with exactly 18506 are those that did not exist in passing data originally
print(bdb_2018.isnull().sum()[bdb_2018.isnull().sum() > 0])

HomeTeamAbbr       18506
VisitorTeamAbbr    18506
Week               18506
Stadium            18506
StadiumType        18506
Turf               18506
GameWeather        19315
Temperature        19508
Humidity           18786
WindSpeed          20053
dtype: int64


In [20]:
## Fill in the columns that need merging
# Method: Group by the game ID, since these values are uniform for the game, replace NaNs with first in group
bdb_2018['HomeTeamAbbr'] = bdb_2018.groupby('GameId')['HomeTeamAbbr'].transform('first')
bdb_2018['VisitorTeamAbbr'] = bdb_2018.groupby('GameId')['VisitorTeamAbbr'].transform('first')
bdb_2018['Week'] = bdb_2018.groupby('GameId')['Week'].transform('first')
bdb_2018['Stadium'] = bdb_2018.groupby('GameId')['Stadium'].transform('first')
bdb_2018['StadiumType'] = bdb_2018.groupby('GameId')['StadiumType'].transform('first')
bdb_2018['Turf'] = bdb_2018.groupby('GameId')['Turf'].transform('first')
bdb_2018['GameWeather'] = bdb_2018.groupby('GameId')['GameWeather'].transform('first')
bdb_2018['Temperature'] = bdb_2018.groupby('GameId')['Temperature'].transform('first')
bdb_2018['Humidity'] = bdb_2018.groupby('GameId')['Humidity'].transform('first')
bdb_2018['WindSpeed'] = bdb_2018.groupby('GameId')['WindSpeed'].transform('first')

In [21]:
## The remaining nulls are for the games that did not contain this info in the rushing data
## Will have to proceed without them
print(bdb_2018.isnull().sum()[bdb_2018.isnull().sum() > 0])

GameWeather    2226
Temperature    2578
Humidity        686
WindSpeed      4029
dtype: int64


In [22]:
## Create features for complete data
bdb_2018.rename(columns={'PossessionTeam':'OffTeam'}, inplace=True)
bdb_2018['OffTeam'] = ['ARI' if ot=='ARZ' else
                       'BAL' if ot=='BLT' else
                       'CLE' if ot=='CLV' else
                       'HOU' if ot=='HST' else
                       ot for ot in bdb_2018['OffTeam']]
bdb_2018['FieldPosition'] = ['ARI' if fp=='ARZ' else
                             'BAL' if fp=='BLT' else
                             'CLE' if fp=='CLV' else
                             'HOU' if fp=='HST' else
                             fp for fp in bdb_2018['FieldPosition']]
# Create DefensiveTeam variable
bdb_2018['DefTeam'] = np.where(bdb_2018['OffTeam'] == bdb_2018['HomeTeamAbbr'],
                               bdb_2018['VisitorTeamAbbr'], bdb_2018['HomeTeamAbbr'])

# Change GameClock to integer of minutes
min_left_qtr = pd.to_datetime(bdb_2018['GameClock'].str[:5])
bdb_2018['Time Left in Quarter'] = min_left_qtr.dt.hour + (min_left_qtr.dt.minute/60)

# Yards to go for first = Distance
# Yards to go for TD = 100-YardLine if OffTeam == FieldPosition, else it is the same as YardLine
bdb_2018['Distance for TD'] = np.where(bdb_2018['OffTeam']==bdb_2018['FieldPosition'],
                                       100 - bdb_2018['YardLine'], bdb_2018['YardLine'])

# Current score in the game
# Difference between OffTeam and DefTeam
# First create "PosTeamScore" and "DefTeamScore", then take the difference
bdb_2018['PosTeamScore'] = np.where(bdb_2018['OffTeam'] == bdb_2018['HomeTeamAbbr'],
                                    bdb_2018['HomeScoreBeforePlay'], bdb_2018['VisitorScoreBeforePlay'])
bdb_2018['DefTeamScore'] = np.where(bdb_2018['DefTeam'] == bdb_2018['HomeTeamAbbr'],
                                    bdb_2018['HomeScoreBeforePlay'], bdb_2018['VisitorScoreBeforePlay'])
bdb_2018['ScoreDifference'] = bdb_2018['PosTeamScore'] - bdb_2018['DefTeamScore']

# Is the possessing team at home?
bdb_2018['PosTeamHome'] = np.where(bdb_2018['OffTeam'] == bdb_2018['HomeTeamAbbr'], 1, 0)

# create indicator if team is in Redzone (<= 20 yds to go from goal)
bdb_2018['Redzone'] = np.where(bdb_2018['Distance for TD'] <= 20, 1, 0)

# create indicator if time left in HALF is <= 2min
bdb_2018['Under2Min'] = np.where(((bdb_2018['Time Left in Quarter']<=2.0) & (bdb_2018['Quarter']==2)|(bdb_2018['Quarter']==4)), 1, 0)

# need to include drive #, timeout data

In [23]:
## Create separate columns for Offensive and Defensive Personnel
# NumQB, NumRB, NumWR, NumTE, NumOL    for Offense
# NumDB, NumLB, NumDL, NumOther        for Defense
def split_personnel(s, offense=True):
    # Split by comma
    splits = s.split(',')
    # Remove whitespaces
    for i in range(len(splits)):
        splits[i] = splits[i].strip()
    
    if offense:
        # Create count of key positions
        qb, rb, wr, te, ol = 0, 0, 0, 0, 0
        # Running subtotal in row, must sum to 11 at the end of for loop
        subtotal = 0
        # Some lists don't have QB present
        qb_listed = False
        for position in splits:
            # Separate by space
            ss = position.split(' ')
            pos = ss[1] # Name of position
            cnt = int(ss[0]) # Number of players at position
            if pos == 'QB':
                qb += cnt # Add count to QB
                subtotal += cnt
                qb_listed = True
            # LB is likely a linebacker lined up as FB/RB
            elif pos in ['RB','LB']:
                rb += cnt
                subtotal += cnt
            # DB is likely a def.back lined up as WR
            elif pos in ['WR','DB']:
                wr += cnt
                subtotal += cnt
            elif pos == 'TE':
                te += cnt
                subtotal += cnt
            else: # the rest are OL
                ol += cnt
                subtotal += cnt
        # Case when not all 11 players were noted at given positions
        # If a QB is not listed, then there was 1 QB on the play
        # If a QB is listed, then assume the rest of the positions are at OL (since RB/FB, TE, WR were always accounted for)
        if subtotal < 11:
            diff = 11 - subtotal
            if not qb_listed:
                qb += 1
                diff -= 1
            ol += diff
            
        return (qb,rb,wr,te,ol) # tuple of counts by position
    
    else: # for defensive personnel, mimic same procedure
          # no need 
        dl, lb, db, other = 0, 0, 0, 0
        for position in splits:
            ss = position.split(' ')
            if ss[1] == 'DL':
                dl += int(ss[0])
            elif ss[1] in ['LB','OL']:
                lb += int(ss[0])
            elif ss[1] in ['DB']:
                db += int(ss[0])
            else:
                other += int(ss[0])
                
        return (dl,lb,db,other)

In [24]:
## Apply function above to data
bdb_2018['OffensePersonnel'] = bdb_2018['OffensePersonnel'].apply(lambda x: split_personnel(x, offense=True))
bdb_2018['DefensePersonnel'] = bdb_2018['DefensePersonnel'].apply(lambda x: split_personnel(x, offense=False))

# Create separate cols for NumQB, NumRB, NumWR, NumTE, NumOL, NumDB, NumLB, NumDL
bdb_2018['NumQB'] = bdb_2018['OffensePersonnel'].apply(lambda x: x[0])
bdb_2018['NumRB'] = bdb_2018['OffensePersonnel'].apply(lambda x: x[1])
bdb_2018['NumWR'] = bdb_2018['OffensePersonnel'].apply(lambda x: x[2])
bdb_2018['NumTE'] = bdb_2018['OffensePersonnel'].apply(lambda x: x[3])
bdb_2018['NumOL'] = bdb_2018['OffensePersonnel'].apply(lambda x: x[4])

bdb_2018['NumDL'] = bdb_2018['DefensePersonnel'].apply(lambda x: x[0])
bdb_2018['NumLB'] = bdb_2018['DefensePersonnel'].apply(lambda x: x[1])
bdb_2018['NumDB'] = bdb_2018['DefensePersonnel'].apply(lambda x: x[2])
bdb_2018['NumDOther'] = bdb_2018['DefensePersonnel'].apply(lambda x: x[3])

In [25]:
bdb_2018.head(5)

,GameId,YardLine,Quarter,GameClock,OffTeam,Down,Distance,FieldPosition,HomeScoreBeforePlay,VisitorScoreBeforePlay,OffenseFormation,OffensePersonnel,DefendersInTheBox,DefensePersonnel,HomeTeamAbbr,VisitorTeamAbbr,Week,Stadium,StadiumType,Turf,GameWeather,Temperature,Humidity,WindSpeed,Type,DefTeam,Time Left in Quarter,Distance for TD,PosTeamScore,DefTeamScore,ScoreDifference,PosTeamHome,Redzone,Under2Min,NumQB,NumRB,NumWR,NumTE,NumOL,NumDL,NumLB,NumDB,NumDOther
0,2018090600,30,1,14:22:00,ATL,2,5,ATL,0.0,0.0,I_FORM,"(1, 2, 2, 1, 5)",7.0,"(4, 2, 5, 0)",PHI,ATL,1.0,Lincoln Financial Field,Outdoor,Grass,Cloudy,81.0,71.0,8.0,run,PHI,14.366667,70,0.0,0.0,0.0,0,0,0,1,2,2,1,5,4,2,5,0
1,2018090600,41,1,13:46:00,ATL,1,10,ATL,0.0,0.0,SINGLEBACK,"(1, 1, 3, 1, 5)",7.0,"(4, 2, 5, 0)",PHI,ATL,1.0,Lincoln Financial Field,Outdoor,Grass,Cloudy,81.0,71.0,8.0,run,PHI,13.766667,59,0.0,0.0,0.0,0,0,0,1,1,3,1,5,4,2,5,0
2,2018090600,6,1,12:15:00,ATL,1,6,PHI,0.0,0.0,SINGLEBACK,"(1, 1, 3, 1, 5)",7.0,"(4, 2, 5, 0)",PHI,ATL,1.0,Lincoln Financial Field,Outdoor,Grass,Cloudy,81.0,71.0,8.0,run,PHI,12.250000,6,0.0,0.0,0.0,0,1,0,1,1,3,1,5,4,2,5,0
3,2018090600,1,1,11:41:00,ATL,2,1,PHI,0.0,0.0,JUMBO,"(1, 2, 0, 3, 5)",10.0,"(6, 3, 2, 0)",PHI,ATL,1.0,Lincoln Financial Field,Outdoor,Grass,Cloudy,81.0,71.0,8.0,run,PHI,11.683333,1,0.0,0.0,0.0,0,1,0,1,2,0,3,5,6,3,2,0
4,2018090600,1,1,10:55:00,ATL,4,1,PHI,0.0,0.0,JUMBO,"(1, 2, 0, 3, 5)",11.0,"(6, 3, 2, 0)",PHI,ATL,1.0,Lincoln Financial Field,Outdoor,Grass,Cloudy,81.0,71.0,8.0,run,PHI,10.916667,1,0.0,0.0,0.0,0,1,0,1,2,0,3,5,6,3,2,0


In [26]:
## drop redundant / unnecessary variables for model training
to_drop = ['GameId', 'YardLine', 'GameClock', 'FieldPosition', 'HomeScoreBeforePlay', 'VisitorScoreBeforePlay',
           'OffensePersonnel', 'DefensePersonnel', 'HomeTeamAbbr', 'VisitorTeamAbbr', 'Stadium', 'PosTeamScore',
           'DefTeamScore']

bdb_2018_final = bdb_2018.drop(to_drop, axis=1)

## one-hot encode following categorical variables:
# PossessionTeam, OffenseFormation, StadiumType, Turf, GameWeather, DefTeam
bdb_2018_final_dummy = pd.get_dummies(data=bdb_2018_final, columns=['OffTeam', 'OffenseFormation','StadiumType','Turf','GameWeather','DefTeam'])

## export final to csv
bdb_2018_final_dummy.to_csv('bdb_2018.csv', index=False)

In [27]:
bdb_2018_final_dummy

,Quarter,Down,Distance,DefendersInTheBox,Week,Temperature,Humidity,WindSpeed,Type,Time Left in Quarter,Distance for TD,ScoreDifference,PosTeamHome,Redzone,Under2Min,NumQB,NumRB,NumWR,NumTE,NumOL,NumDL,NumLB,NumDB,NumDOther,OffTeam_ARI,OffTeam_ATL,OffTeam_BAL,OffTeam_BUF,OffTeam_CAR,OffTeam_CHI,OffTeam_CIN,OffTeam_CLE,OffTeam_DAL,OffTeam_DEN,OffTeam_DET,OffTeam_GB,OffTeam_HOU,OffTeam_IND,OffTeam_JAX,OffTeam_KC,OffTeam_LA,OffTeam_LAC,OffTeam_MIA,OffTeam_MIN,OffTeam_NE,OffTeam_NO,OffTeam_NYG,OffTeam_NYJ,OffTeam_OAK,OffTeam_PHI,OffTeam_PIT,OffTeam_SEA,OffTeam_SF,OffTeam_TB,OffTeam_TEN,OffTeam_WAS,OffenseFormation_EMPTY,OffenseFormation_I_FORM,OffenseFormation_JUMBO,OffenseFormation_PISTOL,OffenseFormation_SHOTGUN,OffenseFormation_SINGLEBACK,OffenseFormation_WILDCAT,StadiumType_Indoor,StadiumType_Outdoor,Turf_Grass,Turf_Turf,GameWeather_Clear,GameWeather_Cloudy,GameWeather_Indoor,GameWeather_Rain,GameWeather_Snow,DefTeam_ARI,DefTeam_ATL,DefTeam_BAL,DefTeam_BUF,DefTeam_CAR,DefTeam_CHI,DefTeam_CIN,DefTeam_CLE,DefTeam_DAL,DefTeam_DEN,DefTeam_DET,DefTeam_GB,DefTeam_HOU,DefTeam_IND,DefTeam_JAX,DefTeam_KC,DefTeam_LA,DefTeam_LAC,DefTeam_MIA,DefTeam_MIN,DefTeam_NE,DefTeam_NO,DefTeam_NYG,DefTeam_NYJ,DefTeam_OAK,DefTeam_PHI,DefTeam_PIT,DefTeam_SEA,DefTeam_SF,DefTeam_TB,DefTeam_TEN,DefTeam_WAS
0,1,2,5,7.0,1.0,81.0,71.0,8.0,run,14.366667,70,0.0,0,0,0,1,2,2,1,5,4,2,5,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,1,1,10,7.0,1.0,81.0,71.0,8.0,run,13.766667,59,0.0,0,0,0,1,1,3,1,5,4,2,5,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,1,1,6,7.0,1.0,81.0,71.0,8.0,run,12.250000,6,0.0,0,1,0,1,1,3,1,5,4,2,5,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,1,2,1,10.0,1.0,81.0,71.0,8.0,run,11.683333,1,0.0,0,1,0,1,2,0,3,5,6,3,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,1,4,1,11.0,1.0,81.0,71.0,8.0,run,10.916667,1,0.0,0,1,0,1,2,0,3,5,6,3,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29770,4,2,2,6.0,16.0,60.0,89.0,5.0,pass,2.316667,67,-12.0,1,0,1,1,1,3,1,5,1,5,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
29771,4,1,10,4.0,16.0,60.0,89.0,5.0,pass,2.000000,60,-12.0,1,0,1,1,1,3,1,5,1,4,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
29772,4,1,10,5.0,16.0,60.0,89.0,5.0,pass,1.683333,43,-12.0,1,0,1,1,1,3,1,5,1,4,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
29773,4,2,10,4.0,16.0,60.0,89.0,5.0,pass,1.616667,43,-12.0,1,0,1,1,1,3,1,5,1,4,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
